## Trained Embedded Layer
- created: 23 MArch 2020

In [7]:
from __future__ import absolute_import, division, print_function, unicode_literals
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import os
from os.path import join
import tensorflow as tf
from tensorflow import keras
from joblib import dump, load
from tensorflow.keras import models
from tensorflow.keras import optimizers
from tensorflow.keras import regularizers
from tensorflow.keras import callbacks
from time import time
from datetime import datetime
import shutil 
import argparse
from functools import partial
import pickle
from IPython.display import Audio

import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.keras import backend as K
from tensorflow.keras import regularizers
from tensorflow.keras.layers import Reshape, GRU, InputLayer, AlphaDropout, Activation, BatchNormalization, Dropout, Flatten, Dense, Bidirectional, LSTM, Conv1D, SpatialDropout1D, Concatenate, Multiply, Add

from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report, confusion_matrix
from sklearn.preprocessing import OneHotEncoder

In [2]:
np_load_old = partial(np.load)
np.load = lambda *a,**k: np_load_old(*a, allow_pickle=True, **k)
tf.random.set_seed(42)

## Load Data 

In [3]:
def prep_data(data_path):

    # load data from preprocess pipeline
    dataset = np.load(join(data_path,'training_subsets.npz'))
    X_train =dataset['X_train']
    z_train= dataset['z_train']
    X_val = dataset['X_val']
    z_val = dataset['z_val']
    X_test = dataset['X_test']
    z_test = dataset['z_test']
    a_test = dataset['audio_test']
    a_val = dataset['audio_val']
    
    print('Train features and labels %s %s'%(str(X_train.shape),str(z_train.shape)))
    print('Validating features and labels %s %s'%(str(X_val.shape),str(z_val.shape)))
    print('Test features and labels %s %s'%(str(X_test.shape),str(z_test.shape)))

    return X_train, X_val, X_test, z_train, z_val, z_test, a_test, a_val

In [4]:
DATA_PATH = '../../data/d_dataset_p2/prep_data_13'
X_train, X_val, X_test, z_train, z_val, z_test, a_test, a_val = prep_data(DATA_PATH)

Train features and labels (102624, 24, 39) (102624,)
Validating features and labels (12568, 24, 39) (12568,)
Test features and labels (12568, 24, 39) (12568,)


In [5]:
print(set(z_test))

{'@', 'u', '9', 'U', 'o', 'A', 'i', 'y', 'E:', 'E', 'O', 'e', '2', 'a'}


### One-hot label 

In [8]:
z_train_oe=z_train.reshape(-1,1)
z_val_oe=z_val.reshape(-1,1)
z_test_oe=z_test.reshape(-1,1)

enc = OneHotEncoder(handle_unknown = 'ignore')
enc.fit(z_train_oe)

z_train_oe = enc.transform(z_train_oe).toarray()
z_test_oe = enc.transform(z_test_oe).toarray()
z_val_oe = enc.transform(z_val_oe).toarray()

dump(enc, 'onehot.joblib')

['onehot.joblib']

### Defined (Between) Embedding Model

In [7]:
PHONETIC_OUTPUT = len(set(z_train))
EMBEDDED_FEATURE = 32

pLSTM = partial(LSTM, kernel_initializer='he_uniform', return_sequences=True)

def embeded_model(input_shape_1,input_shape_2):
    inputs = keras.Input(shape=(input_shape_1,input_shape_2))
    x = pLSTM(EMBEDDED_FEATURE)(inputs)
    x = Flatten()(x)
    outputs = Dense(PHONETIC_OUTPUT, activation='softmax', kernel_initializer='he_uniform')(x)
    return keras.Model(inputs=inputs, outputs=outputs)

In [8]:
embedded = embeded_model(24, 39)
embedded.summary()

Model: "model"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         [(None, 24, 39)]          0         
_________________________________________________________________
lstm (LSTM)                  (None, 24, 32)            9216      
_________________________________________________________________
flatten (Flatten)            (None, 768)               0         
_________________________________________________________________
dense (Dense)                (None, 14)                10766     
Total params: 19,982
Trainable params: 19,982
Non-trainable params: 0
_________________________________________________________________


### Compile Model 

In [9]:
embedded.compile(optimizer='adam',loss=tf.keras.losses.CategoricalCrossentropy(from_logits=True),metrics=['accuracy'])

### Train Model 

In [10]:
PATIENCE = 10
BATCH_SIZE = 256
EPOCHS = 100

early = callbacks.EarlyStopping(monitor='val_loss', min_delta=0, patience=PATIENCE, verbose=1, mode='min', baseline=None, restore_best_weights=False)
callback_list = [early]

In [11]:
history = embedded.fit(X_train,z_train_oe,
    batch_size=BATCH_SIZE,
    epochs=EPOCHS,
    verbose=2,
    validation_data=(X_val,z_val_oe),
    callbacks=callback_list)

Train on 102624 samples, validate on 12568 samples
Epoch 1/100
102624/102624 - 6s - loss: 2.0288 - accuracy: 0.7603 - val_loss: 1.8707 - val_accuracy: 0.8941
Epoch 2/100
102624/102624 - 2s - loss: 1.8942 - accuracy: 0.8695 - val_loss: 1.8419 - val_accuracy: 0.9188
Epoch 3/100
102624/102624 - 2s - loss: 1.8707 - accuracy: 0.8911 - val_loss: 1.8308 - val_accuracy: 0.9300
Epoch 4/100
102624/102624 - 2s - loss: 1.8578 - accuracy: 0.9031 - val_loss: 1.8240 - val_accuracy: 0.9352
Epoch 5/100
102624/102624 - 2s - loss: 1.8499 - accuracy: 0.9104 - val_loss: 1.8214 - val_accuracy: 0.9374
Epoch 6/100
102624/102624 - 2s - loss: 1.8431 - accuracy: 0.9163 - val_loss: 1.8170 - val_accuracy: 0.9416
Epoch 7/100
102624/102624 - 2s - loss: 1.8380 - accuracy: 0.9217 - val_loss: 1.8162 - val_accuracy: 0.9416
Epoch 8/100
102624/102624 - 2s - loss: 1.8331 - accuracy: 0.9261 - val_loss: 1.8152 - val_accuracy: 0.9408
Epoch 9/100
102624/102624 - 2s - loss: 1.8296 - accuracy: 0.9295 - val_loss: 1.8135 - val_acc

In [12]:
embedded.save('between_embedded_32.hdf5')

In [13]:
def evaluate_model(y_pred, label):
    y_pred = enc.inverse_transform(y_pred)
    label = enc.inverse_transform(label)
#     print('Confusion Matrix')
#     print(confusion_matrix(label, y_pred))
    print('Classification Report')
    print(classification_report(label, y_pred))

In [14]:
z_pred = embedded.predict(X_test)
evaluate_model(z_pred, z_test_oe)

Classification Report
              precision    recall  f1-score   support

           2       0.95      0.95      0.95       934
           9       0.97      0.99      0.98       886
           @       0.98      0.96      0.97       903
           A       0.95      0.96      0.95       888
           E       0.94      0.95      0.95       890
          E:       0.95      0.96      0.96       879
           O       0.95      0.94      0.95       964
           U       0.94      0.95      0.94       890
           a       0.97      0.95      0.96       942
           e       0.94      0.96      0.95       894
           i       0.97      0.94      0.95       903
           o       0.93      0.89      0.91       835
           u       0.92      0.95      0.94       895
           y       0.94      0.96      0.95       865

    accuracy                           0.95     12568
   macro avg       0.95      0.95      0.95     12568
weighted avg       0.95      0.95      0.95     12568

